# Boring Pre-req loading data and packages

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch


In [18]:
df = pd.read_csv('/Users/spencerfonbuena/Desktop/AAPL_1hour.txt', header=None, names = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], sep=',', parse_dates=['Date'], index_col=0)

In [19]:
df

,Open,High,Low,Close,Volume
Date,,,,,
2005-01-03 08:00:00,0.9879,0.9925,0.9803,0.9879,5910576
2005-01-03 09:00:00,0.9879,0.9896,0.9598,0.9667,183156512
2005-01-03 10:00:00,0.9670,0.9689,0.9515,0.9589,180934992
2005-01-03 11:00:00,0.9586,0.9659,0.9545,0.9630,80163776
2005-01-03 12:00:00,0.9630,0.9670,0.9591,0.9659,44763040
...,...,...,...,...,...
2023-03-10 15:00:00,148.6600,148.7700,147.6096,148.4800,10525699
2023-03-10 16:00:00,148.4800,149.1600,148.3400,149.0800,8365405
2023-03-10 17:00:00,149.0400,149.0400,148.5000,148.7600,68331


In [16]:
df.to_csv('/Users/spencerfonbuena/Desktop/AAPL_1hour_expanded_test10.txt')

## Method for creating the labels

In [23]:
def create_labels(df):

    A = 0
    C = 0
    labels = np.array([])
    
    print(df)
    for i in range(0, (len(df))):


        #find 1 percent and 2 percent above and below
        #print(df[A])
        one_low = df[A] * .99
        two_low = df[A] * .98
        one_high = df[A] * 1.01
        two_high = df[A] * 1.02

        #print(f'1 low: {one_low} | 2 low: {two_low} | 1 high: {one_high} | 2 high: {two_high}')
        #initialize the label counter
        label_counter = A

        #this is to make sure that once it either enters the "gone up by one percent" or "gone down by 1 percent"
        #it doesn't enter the other while loops
        pathway = 0

        try:
            #look for the instance when the price increases or decreases by 1 percent
            while df[label_counter] >= one_low and df[label_counter] <= one_high:
                label_counter += 1
                #print(df[label_counter])
            #If the price moved up 1 pecent first, this while loop will trigger and check if it is a two to one, or a one to one trade
            while df[label_counter] >= one_low and df[label_counter] <= two_high:
                label_counter += 1
                pathway = 1
                #print(df[label_counter])
            #Check if price has increased two percent
            if df[label_counter] >= two_high:
                labels = np.append(labels, 3)
                pathway = 1
                #print(df[label_counter])
            #check if price has reversed back down to the one percent marker
            if df[label_counter] <= one_low and pathway == 1:
                labels = np.append(labels, 2)
                #print(df[label_counter])
            
            #if the price moved down 1 pecent first, this will check if it is a two to one, or a one to one trade
            while df[label_counter] <= one_high and df[label_counter] >= two_low and pathway != 1:
                label_counter += 1
                pathway = 2
                #print(df[label_counter])
        
            #check if the price has continued down two percent
            if df[label_counter] <= two_low and pathway != 1:
                labels = np.append(labels, 0)
                #print(df[label_counter])
            #check if price reversed back up to the 1 percent above marker
            if df[label_counter] >= one_high and pathway != 1:
                labels = np.append(labels, 1)
                #print(df[label_counter])
            
            #temporarily store the last label that was added to "labels=[]"
            C = labels[-1]

        except:
            break
        #increment the graph by one time interval
        A += 1 

    #Create an array with the last value before the classification algorithm stopped
    array_append = []
    while A < len(df):
        array_append = np.append(array_append, C)
        A += 1
        

    labels = np.append(labels, array_append)
    return labels
df['Labels'] = create_labels(df['Close'])

Date
2005-01-03 08:00:00      0.9879
2005-01-03 09:00:00      0.9667
2005-01-03 10:00:00      0.9589
2005-01-03 11:00:00      0.9630
2005-01-03 12:00:00      0.9659
                         ...   
2023-03-10 15:00:00    148.4800
2023-03-10 16:00:00    149.0800
2023-03-10 17:00:00    148.7600
2023-03-10 18:00:00    148.5100
2023-03-10 19:00:00    148.6000
Name: Close, Length: 68672, dtype: float64


### Moving averages

In [20]:
#50 period moving average
df['50SMA'] = df['Close'].rolling(50).mean()

#200 period moving average
df['200SMA'] = df['Close'].rolling(200).mean()

### RSI

In [21]:
def RSI(df, lookback):
    deltas = np.diff(df)
    seed = deltas[:lookback+1]
    up = seed[seed>= 0].sum()/lookback
    down = -seed[seed < 0].sum()/lookback
    rs = up/down
    rsi = np.zeros_like(df)
    
    for i in range(lookback, len(df)):
        delta = deltas[i-1]

        if delta > 0:
            upval = delta
            downval = 0.
        if delta < 0:
            upval = 0
            downval=abs(delta)
        up = (up * (lookback - 1) + upval) / lookback
        down = (down * (lookback - 1) + downval) / lookback

        rs = up/down
        rsi[i] = 100. - 100./(1. +rs)

    return rsi
df['RSI'] = RSI(df['Close'], 14)
    


### Calculating on balance volume

In [22]:
def onBalanceVolume(df, volume):
    OBV_INTER = 0
    counter = 0
    OBV = np.array([0])
    
    for i in df:
        try:
            if df[counter + 1] > df[counter]:
                OBV_INTER = OBV_INTER + volume[counter]
                OBV = np.append(OBV, OBV_INTER)
            elif df[counter + 1] < df [counter]:
                OBV_INTER = OBV_INTER - volume[counter]
                OBV = np.append(OBV, OBV_INTER)
            elif df[counter + 1] == df[counter]:
                OBV = np.append(OBV, volume[counter])
            counter += 1
        except:
            break
    
    return OBV
df['OBV'] = onBalanceVolume(df['Close'], df['Volume'])

In [6]:
df.shape

(68672, 9)

## Method for windowing the data

In [15]:
def window_dataset(df, window_size):
    A = 0
    window_set = []
    for i in range(len(df) - 100):
        example = df[i: window_size + i]
        window_set.append(np.expand_dims(example, 0))
        if i % 1000 == 0:
            print(i)
    return torch.tensor(np.vstack(window_set)).transpose(-1,-2)
df_expand = window_dataset(df, 100)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000


In [11]:
df_expand

torch.Size([68572, 10, 100])